In [2]:
# 본 트레이닝
from ultralytics import YOLO

# 가장 가벼운 모델 로드
# model = YOLO('yolo26n.pt')

# 마지막 에포크 로드
model = YOLO('runs/detect/train/weights/best.pt')

In [ ]:
# 학습 실행
train_result = model.train(
    data='cv_XIDENT/data.yaml',   # 파일럿 데이터 경로가 적힌 yaml
    name='train_lcls',
    epochs=50,                # 검증용이므로 짧게 설정
    imgsz=640,
    cls=1.5,
    batch=-1,
    device=0,
    workers=10,
    patience=10,
    save_period=5
    # dfl=2.0                # '맞추기 쉬운 샘플'의 비중을 줄이고 '어려운 샘플'에 집중하게 만드는 방식
)

New https://pypi.org/project/ultralytics/8.4.14 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.13  Python-3.11.2 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=2.0, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=f:\Study\cv\cv_XIDENT\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs\detect\train\weights\last.pt, momentum=0.937, mosaic=1.0, 

In [6]:
model = YOLO('runs/detect/train/weights/best.pt')

metrics = model.val(data="cv_XIDENT/data.yaml", conf=0.25, visualize=True)

Ultralytics 8.4.13  Python-3.11.2 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLO26n summary (fused): 122 layers, 2,375,523 parameters, 0 gradients, 5.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 630.7324.6 MB/s, size: 92.4 KB)
val: Scanning F:\Study\cv\dataset_50k\Validation\labels.cache... 14025 images, 528 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 14025/14025  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 877/877 3.6it/s 4:01<0.3s
                   all      14025      49644      0.942      0.943      0.962      0.722
            eye_opened       8156      15036      0.941      0.933      0.968      0.688
            eye_closed       6246      11046       0.92      0.898      0.941      0.585
          mouth_opened       5337       5337      0.963      0.908      0.953      0.717
          mouth_closed       4736       4736      0.888      0.977      0.951      0.699
                  

In [ ]:
from ultralytics.utils.benchmarks import benchmark

# Benchmark on GPU
# benchmark(model="runs/detect/train/weights/best.pt", data="cv_XIDENT/data.yaml", imgsz=640, format='', device=0)

# Benchmark specific export format
benchmark(model="runs/detect/train/weights/best.pt", data="data.yaml", imgsz=640, format="openvino", device="intel:gpu")
benchmark(model="runs/detect/train/weights/best.pt", data="data.yaml", imgsz=640, format="onnx")

Setup complete  (12 CPUs, 15.7 GB RAM, 225.9/237.5 GB disk)

Benchmarks complete for runs/detect/train/weights/best.pt on data.yaml at imgsz=640 (3.75s)
Benchmarks legend:  -  Success  -  Export passed but validation failed  -  Export failed
+----------------------------------------------------------------------------------------+
|     Format   Status   Size (MB)   metrics/mAP50-95(B)   Inference time (ms/im)   FPS |
+========================================================================================+
| 1   ONNX              0.0         -                     -                        -   |
+----------------------------------------------------------------------------------------+



,Format,Status❔,Size (MB),metrics/mAP50-95(B),Inference time (ms/im),FPS
"""1""","""ONNX""","""❌""","""0.0""","""-""","""-""","""-"""


In [11]:
# onnx predict
model_onnx = YOLO('runs/detect/train/weights/best.onnx')

source = 'dataset_50k/Validation/images/045_G1_01_무광원_네비게이션_하품재현_20200929_110850_03518_flip.jpg'

onnx_result = model_onnx.predict(source, conf=0.25, save=True, exist_ok=True, name='onnx_result')

# View results
for r in onnx_result:
    print(r)
    print(r.boxes)

Loading runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.24.1 with CUDAExecutionProvider

image 1/1 f:\Study\cv\dataset_50k\Validation\images\045_G1_01____20200929_110850_03518_flip.jpg: 640x640 2 eye_openeds, 1 mouth_opened, 1 face, 7.0ms
Speed: 0.5ms preprocess, 7.0ms inference, 0.7ms postprocess per image at shape (1, 1, 640, 640)
Results saved to F:\Study\cv\runs\detect\onnx_result
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'eye_opened', 1: 'eye_closed', 2: 'mouth_opened', 3: 'mouth_closed', 4: 'face'}
obb: None
orig_img: array([[[ 13],
        [ 34],
        [ 46],
        ...,
        [ 61],
        [ 70],
        [  0]],

       [[149],
        [ 50],
        [ 25],
        ...,
        [ 65],
        [ 74],
        [  2]],

       [[ 63],
        [ 44],
        [ 38],
        ...,
        [ 68],
        [ 77],
        [  3]],

       ...,


In [15]:
# pt predict
model = YOLO('runs/detect/train/weights/best.pt')

source = 'dataset_50k/Validation/images/045_G1_01_무광원_네비게이션_하품재현_20200929_110850_03518_flip.jpg'

result = model.predict(source, conf=0.25, save=True, exist_ok=True, name='result')

# View results
for r in result:
    # print(r)
    print(r.boxes)


image 1/1 f:\Study\cv\dataset_50k\Validation\images\045_G1_01____20200929_110850_03518_flip.jpg: 640x416 2 eye_openeds, 1 mouth_opened, 1 face, 23.3ms
Speed: 2.5ms preprocess, 23.3ms inference, 0.5ms postprocess per image at shape (1, 1, 640, 416)
Results saved to F:\Study\cv\runs\detect\result
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([4., 2., 0., 0.], device='cuda:0')
conf: tensor([0.9392, 0.8469, 0.7656, 0.7504], device='cuda:0')
data: tensor([[1.7546e+02, 5.7754e+02, 4.6545e+02, 9.3847e+02, 9.3920e-01, 4.0000e+00],
        [2.2056e+02, 7.6053e+02, 3.1451e+02, 8.7732e+02, 8.4687e-01, 2.0000e+00],
        [1.8503e+02, 6.4086e+02, 2.3805e+02, 6.7698e+02, 7.6560e-01, 0.0000e+00],
        [2.8231e+02, 6.1719e+02, 3.5189e+02, 6.5805e+02, 7.5043e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1280, 800)
shape: torch.Size([4, 6])
xywh: tensor([[320.4507, 758.0068, 289.9892, 360.9344],
        [267.5341, 818.9225,  93.9475, 116.7903],
 

In [5]:
from ultralytics import YOLO
import glob

test_photos = glob.glob('test_photo/*.jpg')

model = YOLO('runs/detect/train/weights/best.onnx')

for photo in test_photos:
    result = model.predict(photo, conf=0.25, save=True, exist_ok=True, name='result', project='test_photo/')

Loading runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.24.1 with CUDAExecutionProvider

image 1/1 f:\Study\cv\test_photo\20260212_164314.jpg: 640x640 2 eye_openeds, 1 mouth_opened, 1 face, 9.6ms
Speed: 0.7ms preprocess, 9.6ms inference, 1.6ms postprocess per image at shape (1, 1, 640, 640)
Results saved to F:\Study\cv\runs\detect\test_photo\result

image 1/1 f:\Study\cv\test_photo\20260212_164315.jpg: 640x640 2 eye_openeds, 1 mouth_opened, 1 face, 12.7ms
Speed: 0.7ms preprocess, 12.7ms inference, 0.7ms postprocess per image at shape (1, 1, 640, 640)
Results saved to F:\Study\cv\runs\detect\test_photo\result

image 1/1 f:\Study\cv\test_photo\20260212_164316.jpg: 640x640 2 eye_openeds, 2 mouth_openeds, 1 face, 10.6ms
Speed: 0.7ms preprocess, 10.6ms inference, 1.7ms postprocess per image at shape (1, 1, 640, 640)
Results saved to F:\Study\cv\runs\detect\test_photo\result

image 1/1 f:\Study\cv\test_photo\20260212_164317.jpg: 640x640 2 eye_openeds, 1

In [2]:
model = YOLO('runs/detect/train/weights/best.pt')

model_onnx_320 = model.export(format='OpenVINO', name='best_OpenVINO')

Ultralytics 8.4.13  Python-3.11.2 torch-2.6.0+cu124 CPU (12th Gen Intel Core i5-12400F)
YOLO26n summary (fused): 122 layers, 2,375,523 parameters, 0 gradients, 5.1 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 1, 640, 640) BCHW and output shape(s) (1, 300, 6) (5.1 MB)
requirements: Ultralytics requirement ['openvino>=2024.0.0'] not found, attempting AutoUpdate...
   ---------------------------------------- 0.0/41.8 MB ? eta -:--:--
   ------- -------------------------------- 7.3/41.8 MB 41.2 MB/s eta 0:00:01
   ----------------- ---------------------- 18.4/41.8 MB 48.2 MB/s eta 0:00:01
   --------------------------- ------------ 29.1/41.8 MB 49.9 MB/s eta 0:00:01
   ---------------------------------------  41.7/41.8 MB 53.0 MB/s eta 0:00:01
   ---------------------------------------- 41.8/41.8 MB 53.2 MB/s  0:00:00

   ---------------------------------------- 0/2 [openvino-telemetry]
   -------------------- ------------------- 1/2 [openvino]
  